<a href="https://colab.research.google.com/github/Knerubon/pro-is-ex1/blob/main/Copy_of_G6_trend_forecast_stock_price_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary packages
import pandas as pd
import matplotlib.pyplot as plt

# Set plot format
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

# Set float format for Pandas display
pd.options.display.float_format = '{:,.2f}'.format

# Install missing package if needed
!pip install --upgrade starfishX
!pip install mplfinance
# Import the installed package
import starfishX as sx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached starfishX-0.155555-py3-none-any.whl (119 kB)
  Using cached mplfinance-0.12.9b7-py3-none-any.whl (73 kB)
  Using cached starfishX-0.155554-py3-none-any.whl (119 kB)
  Using cached starfishX-0.155553-py3-none-any.whl (119 kB)
  Using cached starfishX-0.155552-py3-none-any.whl (119 kB)
  Using cached starfishX-0.155551-py3-none-any.whl (118 kB)
  Using cached starfishX-0.155550-py3-none-any.whl (118 kB)
  Using cached starfishX-0.155549-py3-none-any.whl (118 kB)
  Using cached starfishX-0.155547-py3-none-any.whl (117 kB)
  Using cached starfishX-0.155546-py3-none-any.whl (117 kB)
  Using cached starfishX-0.155545-py3-none-any.whl (117 kB)
  Using cached starfishX-0.155544-py3-none-any.whl (117 kB)
  Using cached starfishX-0.155543-py3-none-any.whl (117 kB)
  Using cached starfishX-0.155542-py3-none-any.whl (117 kB)
  Using cached starfishX-0.155541-py3-none-any.whl (114 kB)
 

In [ ]:
!pip install --user starfishX

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Import the installed package
import starfishX as sx

In [ ]:
pd.set_option('display.max_columns', 500)

# Work flow
1. Get data
2. Cleansing & transformation:
    
    2.1. extract feature and label

    2.2. one hot

    2.3. train-test split

3. Prediction model
4. Visualize
5. Feature engineer
6. Setup pipeline

## 1. Get data


### ใช้ฟังก์ชั่น ```getFinanceRatio()``` เพื่อดึงข้อมูลทางการเงินของหุ้นที่สนใจ

ถ้าใส่หุ้นตัวเดียวจะได้สัดส่วนทางการเงินย้อนหลัง 4 ช่วงเวลา ถ้าใส่ list ของรายชื่อหุ้น จะได้สัดส่วนทางการเงินล่าสุดของหุ้นที่อยู่ใน list

In [ ]:
# Create get_stock function to pull stock data
def get_stock(stock, startdate,end_date):

  df = sx.loadHistData_v2(symbol=stock, start=startdate , end=end_date)

  return df

In [ ]:
stock_symbol = 'AMZN'
start_date = '2016-01-01'
end_date = '2016-01-01'
df = get_stock(stock_symbol,start_date,end_date)
df

TypeError: ignored

In [ ]:
# Display dataframe information
df.info()

NameError: ignored

In [ ]:
# Display stock price statistic information
df.describe()

In [ ]:
# A quick look at stock price
df.CLOSE.plot(figsize=(15,5))
plt.title('Symbol: {}'.format(stock_symbol))
plt.grid()
plt.legend();

## 2. Cleansing & transformation

### 2.1. Get feature and target dataframe

In [ ]:
df.head()

In [ ]:
# Create get_X_y function to get dependent (y) and independent (X) variables
def get_df(dataframe):

  # Extract time series variable
  df = dataframe.reset_index()[['Date']]
  df['Year'] = [df.iloc[i, 0].year for i in range(len(df))]
  df['Month'] = [df.iloc[i, 0].month for i in range(len(df))]
  df['Day'] = [df.iloc[i, 0].day for i in range(len(df))]
  df['EnglishDay'] = df.Date.dt.strftime('%a')

  # Set categorical variable
  df = df.astype({'Month':'category',
                'Day':'category',
                'EnglishDay':'category'}).set_index('Date')

  # Get OHLC
  df = df.merge(dataframe, left_index=True, right_index=True)
  df['ema_5'] = df['OPEN'].ewm(span=5, adjust=False).mean()
  df['ema_10'] = df['OPEN'].ewm(span=10, adjust=False).mean()
  df['BS_diff'] = (df['ema_5'] - df['ema_10']).apply(lambda x: 'Buy' if x>0 else 'Sell').astype('category')


  return df

In [ ]:
df = get_df(df)

In [ ]:
df.head()

### 2.2. Get one-hot encoder

In [ ]:
# Create one_hot_encoder function
def one_hot_encoder(dataframe):

  # Select category columns
  cat_df = dataframe.select_dtypes(include=['category']).columns.to_list()

  # Convert to one-hot dataframe
  one_hot_df = pd.get_dummies(dataframe, columns=cat_df)

  return one_hot_df

In [ ]:
X_dummie = one_hot_encoder(df)
X_dummie

In [ ]:
X_dummie.info()

In [ ]:
# Create get_X_y function to get dependent (y) and independent (X) variables
def get_X_y(dataframe):

  # Extract time series variable
  X = dataframe.copy()

  # Extract dependent variable
  y = dataframe[['OPEN']].copy()

  return X, y

In [ ]:
# Test get_X_y function
X, y = get_X_y(X_dummie)

# Display 10 rows of X
X.head()

In [ ]:
# Create get_X_y function to get dependent (y) and independent (X) variables
def get_X2(dataframe):

  X2 = dataframe.loc[:, ~dataframe.columns.str.startswith('PV')]
  X2 = X2.iloc[-1:,:]
  X2.columns = ['PV' + str(col) for col in X2.columns]


  return X2

### 2.3. Split train and test dataframe

In [ ]:
# Create stock_train_test_split function
def stock_train_test_split(dataframe, y, trainsize=0.8):

  import math

  train_ratio = trainsize
  train_size = math.ceil(len(dataframe) * train_ratio)

  # Get train dataframe
  X_train = dataframe.iloc[:train_size, :]
  y_train = y.iloc[:train_size, :]

  # Get train dataframe
  # X_test = dataframe.iloc[train_size:, :]
  X_test = dataframe.copy()
  y_test = y.iloc[train_size:, :]

  # Drop label from independent variable
  col_drop = ['OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOLUME']
  X_train = X_train.drop(col_drop, axis=1)
  X_test = X_test.drop(col_drop, axis=1)



  return X_train, X_test, y_train, y_test

In [ ]:
# Test stock_train_tes_split function
trainsize = 0.8
X_train, X_test, y_train, y_test = stock_train_test_split(X_dummie, y, trainsize=trainsize)
X_train = X_train.drop('Year', axis=1)
X_test = X_test.drop('Year', axis=1)

In [ ]:
X_train

In [ ]:
# Display 10 rows of X_train
# len(X_train)
display(len(X_train))
X_train.head()

In [ ]:
X_train.tail()

In [ ]:
# Display 10 rows of X_test
display(len(X_test))
X_test.head()

In [ ]:
# Display 10 rows of y_train
y_train.tail()

In [ ]:
# Display 10 rows of y_test
y_test.head()

## 3. Prediction model

### 3.1. Model

In [ ]:
# Create model_fit function for setting prediction model
def model_fit(X_train, y_train):

  # import sci-kit learn
  from sklearn.linear_model import LinearRegression

  # Model fitting with normalization
  model = LinearRegression(normalize=True).fit(X_train, y_train)

  return model

In [ ]:
# Train model with X_train and y_train
model = model_fit(X_train, y_train)

In [ ]:
# Test set prediction
y_hat = model.predict(X_test)

In [ ]:
y_hat[:10]

In [ ]:
y_train.head()

### 3.2. Error

In [ ]:
def rmse(y_test, y_hat):
  # Root mean squared error
  from sklearn.metrics import mean_squared_error
  from math import sqrt

  y_hat_test = y_hat[-len(y_test):]
  rms = sqrt(mean_squared_error(y_test, y_hat_test))
  return rms

In [ ]:
# rms for test dataframe
rmse(y_test, y_hat)

## 4. Visualization

In [ ]:
# Create visualize function to display result
def visualize(X, y, y_hat, symbol, feat_eng=False):

  # Set figure size
  plt.figure(figsize=(10, 5))

  # Plot actual price
  plt.plot(X.index, y, label='Actual Price')

  # Plot predicted price
  plt.plot(X.index, y_hat, label='Predicted Price')

  # Set title
  plt.title('Stock price: {} [{}]'.format(symbol, feat_eng) )

  # Set x-label
  plt.xlabel('Period')

  # Set y-label
  plt.ylabel('Price [THB]')

  # Plot grid
  plt.grid()

  # Display legend
  plt.legend()

  # Display graph
  plt.show()

In [ ]:
visualize(X, y, y_hat, stock_symbol)

## 5. Feature engineer: Incorporate previous day price and volume features

In [ ]:
import numpy as np
# Create feature_engineer function to extract new features
def feature_engineer(dataframe, feat_eng=False):

  if feat_eng:
    dataframe['PVMonth'] = dataframe[['Month']].shift(periods=6)
    dataframe['PVDay'] = dataframe[['Day']].shift(periods=6)
    dataframe['PVEnglishDay'] = dataframe[['EnglishDay']].shift(periods=6)
    dataframe['PVOPEN'] = dataframe[['OPEN']].shift(periods=6, fill_value=0)
    dataframe['PVHIHG'] = dataframe[['HIGH']].shift(periods=6, fill_value=0)
    dataframe['PVLOW'] = dataframe[['LOW']].shift(periods=6, fill_value=0)
    dataframe['PVCLOSE'] = dataframe[['CLOSE']].shift(periods=6, fill_value=0)
    dataframe['PVVOLUME'] = dataframe[['VOLUME']].shift(periods=6, fill_value=0)
    dataframe['PVema_5'] = dataframe[['ema_5']].shift(periods=6, fill_value=0)
    dataframe['PVema_10'] = dataframe[['ema_10']].shift(periods=6, fill_value=0)
    dataframe['PVBS_diff'] = dataframe[['BS_diff']].shift(periods=6)

    dataframe = dataframe.iloc[10:,1:]
    dataframe['PVMonth'] = dataframe['PVMonth'].astype('int')
    dataframe['PVDay'] = dataframe['PVDay'].astype('int')

    dataframe['PVEnglishDay'] = dataframe['PVEnglishDay'].astype('category')
    dataframe['PVBS_diff'] = dataframe['PVBS_diff'].astype('category')
    dataframe['PVMonth'] = dataframe['PVMonth'].astype('category')
    dataframe['PVDay'] = dataframe['PVDay'].astype('category')
    return dataframe

In [ ]:
# Test our feature_engineer function
X_engineered = feature_engineer(df, feat_eng=True)
X_engineered

In [ ]:
X_engineered.info()

## 6. Setup our pipeline

In [ ]:
# Create main function to run pipeline
def main(stock, startdate='2016-01-01', feat_eng=True):

  # Get stock
  df = get_stock(stock, startdate)
  df = get_df(df)

  # Get engineered features
  X_engineered = feature_engineer(df, feat_eng=feat_eng)

  # One-hot encoder
  X_eng_dummie = one_hot_encoder(X_engineered)

  X, y = get_X_y(X_eng_dummie)


  # Train test splitting
  X_train, X_test, y_train, y_test = stock_train_test_split(X,
                                                            y,
                                                            trainsize=0.8)

  X_train = X_train.loc[:, X_train.columns.str.startswith('PV')]
  X_test = X_test.loc[:, X_test.columns.str.startswith('PV')]

  # Prediction model
  model = model_fit(X_train, y_train)

  # Prediction
  y_hat = model.predict(X_test)

# RMS
  RMS = rmse(y_test, y_hat)
  X2 = get_X2(X_eng_dummie)
  y_pred = model.predict(X2)


  return RMS, y_pred

## Test our pipeline

In [ ]:
# Set all parameters
stock_symbol = 'BANPU'
#stock_symbol = ['BANPU', 'SCB', 'KBANK']
start_date = '2015-01-01'
feat_eng = True

RMS, y_pred = main(stock=stock_symbol, startdate=start_date, feat_eng=feat_eng)
print(RMS)
print(y_pred)

In [ ]:
#List all stock result

StockSearch = pd.DataFrame({'Stock': [], 'Price': [], 'PricePredict': [], 'RMS': []})
StockSearch

In [ ]:
stock_symbol = ['PTT',
'SCB',
'SCC',
'BTS',
'M']

for i in stock_symbol:
    datasearch = sx.loadHistData(i, start='2020-09-11',OHLC=True)
    datasearch = datasearch['CLOSE']
    datasearch1 = i
    # datasearch2 = datasearch.items()
    datasearch2 =datasearch.iloc[-1]
    RMS, y_pred = main(stock=i, startdate=start_date, feat_eng=feat_eng)
    DataResult1 = y_pred
    DataResult2 = RMS


    StockSearch = StockSearch.append({'Stock': datasearch1, 'Price': datasearch2, 'PricePredict': DataResult1, 'RMS': DataResult2}, ignore_index=True)

In [ ]:
StockSearch

In [ ]:
StockSearch['diff'] = (StockSearch['PricePredict'].astype(float))-(StockSearch['Price'].astype(float))
StockSearch['%Change'] = StockSearch['diff']/StockSearch['Price']
StockConclusion = StockSearch.sort_values(by='%Change', ascending = False)
StockConclusion

# Test Algorithm

In [ ]:
%%capture
!pip install pycaret==2.0

In [ ]:
from pycaret.utils import enable_colab
enable_colab()

In [ ]:
# Create main function to run pipeline
def get_Xytrain(stock, startdate='2016-01-01', feat_eng=True):

  # Get stock
  df = get_stock(stock, startdate)
  df = get_df(df)

  # Get engineered features
  X_engineered = feature_engineer(df, feat_eng=feat_eng)

  # One-hot encoder
  X_eng_dummie = one_hot_encoder(X_engineered)

  X, y = get_X_y(X_eng_dummie)

  return X, y

In [ ]:
stock_symbol = 'BANPU'
#stock_symbol = ['BANPU', 'SCB', 'KBANK']
start_date = '2015-01-01'
feat_eng = True

X, y = get_Xytrain(stock=stock_symbol, startdate=start_date, feat_eng=feat_eng)

In [ ]:
data = X.merge(y, left_index=True, right_index=True)
data['PVlabel'] = data['OPEN_x']
data = data.loc[:, data.columns.str.startswith('PV')]
data.head()

In [ ]:
from pycaret.regression import *
exp_reg101 = setup(data = data, target = 'PVlabel', session_id=123)

In [ ]:
# ransac = random sample consensus
best = compare_models()#blacklist = ['ransac'])

In [ ]:
####ไว้รันหุ้น Set100
StockSearch = pd.DataFrame({'Stock': [], 'Price': [], 'PricePredict': [], 'RMS': []})
stock_symbol = [
'AAV',
'ADVANC',
'AEONTS',
'AMATA',
'AOT',
'AP',
'AWC',
'BANPU',
'BBL',
'BCH',
'BCP',
'BCPG',
'BDMS',
'BEC',
'BEM',
'BGRIM',
'BH',
'BJC',
'BPP',
'BTS',
'CBG',
'CENTEL',
'CHG',
'CK',
'CKP',
'COM7',
'CPALL',
'CPF',
'CPN',
'DOHOME',
'DTAC',
'EA',
'EGCO',
'EPG',
'ERW',
'ESSO',
'GFPT',
'GLOBAL',
'GPSC',
'GULF',
'GUNKUL',
'HANA',
'HMPRO',
'INTUCH',
'IRPC',
'IVL',
'JAS',
'KBANK',
'KCE',
'KKP',
'KTB',
'KTC',
'LH',
'MAJOR',
'MEGA',
'MINT',
'ORI',
'OSP',
'PLANB',
'PRM',
'PSH',
'PTTEP',
'PTTGC',
'QH',
'RATCH',
'RBF',
'RS',
'SAWAD',
'SCB',
'SCC',
'SGP',
'SIRI',
'SPALI',
'SPRC',
'STA',
'STEC',
'SUPER',
'TASCO',
'TCAP',
'TISCO',
'TKN',
'TMB',
'TOP',
'TQM',
'TRUE',
'TTW',
'TU',
'TVO',
'VGI',
'WHA',
'WHAUP'
]


for i in stock_symbol:
    datasearch = sx.loadHistData(i, start='2020-09-11',OHLC=True)
    datasearch = datasearch['CLOSE']
    datasearch1 = i
    # datasearch2 = datasearch.items()
    datasearch2 =datasearch.iloc[-1]
    RMS, y_pred = main(stock=i, startdate=start_date, feat_eng=feat_eng)
    DataResult1 = y_pred
    DataResult2 = RMS


    StockSearch = StockSearch.append({'Stock': datasearch1, 'Price': datasearch2, 'PricePredict': DataResult1, 'RMS': DataResult2}, ignore_index=True)


In [ ]:
StockSearch['diff'] = (StockSearch['PricePredict'].astype(float))-(StockSearch['Price'].astype(float))
StockSearch['%Change'] = StockSearch['diff']/StockSearch['Price']
StockConclusion = StockSearch.sort_values(by='%Change', ascending = False)
StockConclusion

In [ ]:
StockConclusion.head(10)